In [13]:
import os 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from glob import glob 
import json 

exp_list= glob('./*')

metrics = ['Image AUC', 'Pixel AUC', 'Image AP', 'Pixel AP']
df = pd.DataFrame()
for exp_path in exp_list:
    for log_path in glob(os.path.join(exp_path, 'final_results.csv')):
        temp_df = pd.read_csv(log_path)
        # metrics 컬럼을 float로 변환
        for m in metrics:
            if m in temp_df.columns:
                temp_df[m] = pd.to_numeric(temp_df[m], errors='coerce')
        temp_df['exp_name'] = os.path.basename(exp_path)
        df = pd.concat([df, temp_df], ignore_index=True)


In [14]:
# 'Class Name'이 'Overall'이 아닌 행들만 남기고,
# exp_name에서 dataset을 추출하여 dataset별로 실험들을 비교합니다.
metrics = ['Image AUC',  'Pixel AP']

df_filtered = df.copy()
for m in metrics:
    df_filtered[m] = pd.to_numeric(df_filtered[m], errors='coerce')

# exp_name에서 dataset 추출 (첫 번째 하이픈 또는 언더스코어로 분리)
def extract_dataset(exp_name):
    """exp_name에서 dataset을 추출합니다."""
    if pd.isna(exp_name):
        return 'Unknown'
    exp_str = str(exp_name)
    # 하이픈으로 분리 시도
    if '-' in exp_str:
        return exp_str.split('-')[0]
    # 언더스코어로 분리 시도
    elif '_' in exp_str:
        return exp_str.split('_')[0]
    # 분리자가 없으면 전체를 dataset으로 간주
    else:
        return exp_str

df_filtered['dataset'] = df_filtered['exp_name'].apply(extract_dataset)

# dataset별 exp_name별 전체 평균 (각 실험별 전체 평균)
df_exp_mean = df_filtered.groupby(['dataset', 'exp_name'])[metrics].mean().reset_index()

# Average 컬럼 추가 (Image AUC, Pixel AP의 평균)
df_exp_mean['Average'] = df_exp_mean[metrics].mean(axis=1)

# Average 기준 내림차순 정렬 후 상위 10개 실험만 남기기
df_top10 = df_exp_mean.sort_values('Average', ascending=False).groupby('dataset').head(15).reset_index(drop=True)

# dataset별 클래스별 결과 비교 (피벗 테이블: index=Class Name, columns=exp_name, grouped by dataset)
datasets = df_filtered['dataset'].unique()
for dataset in sorted(datasets):
    print(f"\n{'='*80}")
    print(f"Dataset: {dataset}")
    print(f"{'='*80}")
    
    # 해당 dataset의 데이터만 필터링
    df_dataset = df_filtered[df_filtered['dataset'] == dataset]
    
    # 해당 dataset의 exp_name별 클래스별 결과
    df_class_compare = df_dataset.pivot_table(index='Class Name', columns='exp_name', values=metrics)
    
    
    print(f"\n[{dataset} - 실험별 전체 평균 (Average 상위 10개만)]")
    display(df_top10[df_top10['dataset'] == dataset])
    
    print(f"\n[{dataset} - 클래스별 비교]")
    display(df_class_compare)



Dataset: MPDD

[MPDD - 실험별 전체 평균 (Average 상위 10개만)]


,dataset,exp_name,Image AUC,Pixel AP,Average
15,MPDD,MPDD-WRN50-60ep-lr2e4-dia4,0.901883,0.288983,0.595433



[MPDD - 클래스별 비교]


,Image AUC,Pixel AP
exp_name,MPDD-WRN50-60ep-lr2e4-dia4,MPDD-WRN50-60ep-lr2e4-dia4
Class Name,,
bracket_black,0.7427,0.0108
bracket_brown,1.0000,0.0133
bracket_white,0.7844,0.0020
connector,0.9929,0.2948
metal_plate,1.0000,0.9003
tubes,0.8913,0.5127



Dataset: MVTec

[MVTec - 실험별 전체 평균 (Average 상위 10개만)]


...

Image AUC                                               \
exp_name   MVTec-ViT-60ep MVTec-WRN50-60ep-lr2e4-dia4 MVTec-WRN50-80ep   
Class Name                                                               
bottle             1.0000                      1.0000           1.0000   
cable              0.9350                      0.9884           0.9933   
capsule            0.7495                      0.9609           0.9569   
carpet             0.9759                      0.9835           0.9835   
grid               0.9315                      0.9983           0.9908   
hazelnut           0.9614                      0.9993           1.0000   
leather            1.0000                      1.0000           1.0000   
metal_nut          0.9804                      1.0000           1.0000   
pill               0.8718                      0.9888           0.9839   
screw              0.4122                      0.9094           0.9035   
tile               1.0000                      1.0000           1.0000   
toothbrush         0.8972                      0.8861           0.9000   
transistor         0.8154                      0.9950           1.0000   
wood               0.9675                      0.9842           0.9904   
zipper             0.9315                      0.9961           0.9916   

                                                                        \
exp_name   MVTec-WRN50-Ablation-wo_Adapter MVTec-WRN50-Ablation-wo_DIA   
Class Name                                                               
bottle                              1.0000                      1.0000   
cable                               0.9846                      0.9783   
capsule                             0.8672                      0.8077   
carpet                              0.9876                      0.9912   
grid                                0.9825                      0.9758   
hazelnut                            1.0000                      0.9996   
leather                             1.0000                      1.0000   
metal_nut                           0.9995                      0.9985   
pill                                0.9716                      0.9626   
screw                               0.7530                      0.6661   
tile                                1.0000                      1.0000   
toothbrush                          0.8806                      0.8833   
transistor                          1.0000                      0.9971   
wood                                0.9895                      0.9877   
zipper                              0.9898                      0.9709   

                                                                          \
exp_name   MVTec-WRN50-Ablation-wo_LoRA MVTec-WRN50-Ablation-wo_PosEmbed   
Class Name                                                                 
bottle                           1.0000                           1.0000   
cable                            0.9918                           0.9888   
capsule                          0.9513                           0.9509   
carpet                           0.9868                           0.9900   
grid                             0.9875                           0.9875   
hazelnut                         1.0000                           1.0000   
leather                          1.0000                           1.0000   
metal_nut                        1.0000                           1.0000   
pill                             0.9825                           0.9831   
screw                            0.9006                           0.8817   
tile                             1.0000                           1.0000   
toothbrush                       0.9111                           0.9000   
transistor                       1.0000                           0.9962   
wood                             0.9912                           0.9860   
zipper                           0.9932                   


Dataset: VISA

[VISA - 실험별 전체 평균 (Average 상위 10개만)]


,dataset,exp_name,Image AUC,Pixel AP,Average
16,VISA,VISA-WRN50-LoRA128-DIA6-Combined,0.856558,0.276133,0.566346
17,VISA,VISA-WRN50-60ep-lr2e4-dia4,0.837833,0.287775,0.562804
18,VISA,VISA-WRN50-DIA6-80ep,0.837575,0.275025,0.556300
19,VISA,VISA-WRN50-80ep-lr3e4,0.827192,0.269783,0.548488
20,VISA,VISA-ViT-60ep,0.880125,0.198217,0.539171
21,VISA,VISA-WRN50-LoRA128-80ep,0.820200,0.257133,0.538667



[VISA - 클래스별 비교]


Image AUC                                                   \
exp_name   VISA-ViT-60ep VISA-WRN50-60ep-lr2e4-dia4 VISA-WRN50-80ep-lr3e4   
Class Name                                                                  
candle            0.8932                     0.8843                0.8200   
capsules          0.8692                     0.6743                0.6967   
cashew            0.9726                     0.8794                0.8474   
chewinggum        0.9154                     0.9630                0.9542   
fryum             0.9262                     0.9360                0.9526   
macaroni1         0.8419                     0.7609                0.7375   
macaroni2         0.6490                     0.6597                0.6748   
pcb1              0.9234                     0.8428                0.8393   
pcb2              0.8767                     0.7309                0.7369   
pcb3              0.8121                     0.7845                0.7676   
pcb4              0.9492                     0.9710                0.9495   
pipe_fryum        0.9326                     0.9672                0.9498   

                                                         \
exp_name   VISA-WRN50-DIA6-80ep VISA-WRN50-LoRA128-80ep   
Class Name                                                
candle                   0.8439                  0.8304   
capsules                 0.7070                  0.6800   
cashew                   0.8452                  0.8352   
chewinggum               0.9536                  0.9468   
fryum                    0.9502                  0.9414   
macaroni1                0.7569                  0.7428   
macaroni2                0.6813                  0.6657   
pcb1                     0.8394                  0.8052   
pcb2                     0.7790                  0.7411   
pcb3                     0.7818                  0.7548   
pcb4                     0.9584                  0.9516   
pipe_fryum               0.9542                  0.9474   

                                                 Pixel AP  \
exp_name   VISA-WRN50-LoRA128-DIA6-Combined VISA-ViT-60ep   
Class Name                                                  
candle                               0.8619        0.0167   
capsules                             0.7077        0.2378   
cashew                               0.8686        0.6154   
chewinggum                           0.9574        0.1976   
fryum                                0.9598        0.2712   
macaroni1                            0.7866        0.0226   
macaroni2                            0.7078        0.0062   
pcb1                                 0.8693        0.4606   
pcb2                                 0.8066        0.0891   
pcb3                                 0.8187        0.1302   
pcb4                                 0.9681        0.1419   
pipe_fryum                           0.9662        0.1893   

                                                             \
exp_name   VISA-WRN50-60ep-lr2e4-dia4 VISA-WRN50-80ep-lr3e4   
Class Name                                                    
candle                         0.1481                0.1627   
capsules                       0.2121                0.1908   
cashew                         0.4962                0.4197   
chewinggum                     0.2754                0.3227   
fryum                          0.4024                0.4233   
macaroni1                      0.0655                0.0625   
macaroni2                      0.0083                0.0078   
pcb1                           0.6797                0.5381   
pcb2                           0.0985                0.0905   
pcb3                           0.2636                0.2617   
pcb4                           0.2805                0.2550   
pipe_fryum                     0.5230                0.5026   

                                                         \
exp_name   VISA-WRN50-DIA6-80ep VISA-WRN50-LoRA128-8

In [15]:
from glob import glob 

import os

dirs = [d for d in glob('*') if os.path.isdir(d)]

dirs_with_csv = []
dirs_without_csv = []

for d in dirs:
    csv_path = os.path.join(d, 'final_results.csv')
    if os.path.isfile(csv_path):
        dirs_with_csv.append(d)
    else:
        dirs_without_csv.append(d)

print(f"디렉토리 내에 Final_results.csv가 있는 곳:\n{dirs_with_csv}")
print(f"디렉토리 내에 Final_results.csv가 없는 곳:\n{dirs_without_csv}")

디렉토리 내에 Final_results.csv가 있는 곳:
['MVTec-WRN50-DIA6-ScaleCtxK5', 'MVTec-WRN50-TopK7-TailW0.5', 'MVTec-WRN50-TailTopK10', 'MVTec-WRN50-Ablation-wo_LoRA', 'MVTec-WRN50-DIA8-lr3e-4', 'MVTec-WRN50-LogdetReg1e-4-ScaleCtxK5', 'MVTec-WRN50-TailTopK3', 'MVTec-WRN50-LogdetReg1e-4-DIA8', 'MVTec-WRN50-TailW0.8-TopK5-TailTopK3-ScaleK5', 'MVTec-WRN50-TopK3-TailW0.5-LogdetReg1e-4-ScaleCtxK5', 'VISA-WRN50-60ep-lr2e4-dia4', 'MVTec-WRN50-DIA4-TopK10', 'MVTec-WRN50-TopK3-TailW0.5-LogdetReg1e-4', 'MVTec-WRN50-LogdetReg1e-4', 'MVTec-WRN50-SpatialCluster', 'VISA-ViT-60ep', 'MVTec-WRN50-Ablation-wo_Adapter', 'MVTec-WRN50-LoRA256', 'MVTec-WRN50-TopK5-TailW0.5-lr3e-4', 'MVTec-WRN50-SpatialCtxK5', 'MVTec-WRN50-lr3e-4', 'MVTec-WRN50-Ablation-wo_ScaleCtx', 'MVTec-WRN50-LogdetReg1e-4-DIA6', 'MVTec-WRN50-Balanced-Combo2', 'MVTec-WRN50-TailW0.65-TailTopK3-TopK5-LogdetReg1e-4', 'MVTec-WRN50-TopK5-TailW0.5-LogdetReg1e-4-80ep', 'MVTec-WRN50-TopK5-TailW0.5-LogdetReg1e-4', 'VISA-WRN50-LoRA128-80ep', 'VISA-WRN50-LoRA128-

In [16]:
['MVTec-WRN50-DIA6-ScaleCtxK5', 'MVTec-WRN50-LogdetReg1e-4-DIA6', 'MVTec-WRN50-Ultimate-Combo1', 'MVTec-WRN50-TopK5-TailW0.5-DIA6']

['MVTec-WRN50-DIA6-ScaleCtxK5',
 'MVTec-WRN50-LogdetReg1e-4-DIA6',
 'MVTec-WRN50-Ultimate-Combo1',
 'MVTec-WRN50-TopK5-TailW0.5-DIA6']